In [1]:
import pandas as pd
import os
import glob

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Remove existing records, in case changes result in changed names for new records. 

In [2]:
for file in glob.glob('*.md'):
    os.remove(file)

Create updated grant records

In [34]:
# 2023 output
#grants = pd.read_csv('nrpf-grants-data.csv', sep=',', header = 0)
# 2025 output
grants = pd.read_csv('nrpf-grants-data-202505.csv', sep = ',', header = 0)

In [35]:
grants.shape

(32, 19)

In [36]:
print(grants.dtypes)

Award_Year_Reported_on_990    float64
Amount                         object
Recipient                      object
Description_990                object
Source                         object
Notes                          object
Recipient_EIN                  object
City                           object
State                          object
Type                           object
Activity                       object
Output                         object
Title_web                      object
Description_web                object
Project_director               object
Project_extent_preserved       object
Project_date_range             object
Project_URL                   float64
hours_preserved               float64
dtype: object


In [37]:
# convert floats to strings
grants['Award_Year_Reported_on_990'] = grants['Award_Year_Reported_on_990'].astype('Int64').astype(str)
grants['Project_URL'] = grants['Project_URL'].astype('Int64').astype(str)

In [38]:
# filter out zero amounts & null amounts & a specific amount
grants = grants[(grants.Amount!="$0.00") & (grants.Amount.notnull()) & (grants.Amount!='$253,382.00')]
grants

,Award_Year_Reported_on_990,Amount,Recipient,Description_990,Source,Notes,Recipient_EIN,City,State,Type,Activity,Output,Title_web,Description_web,Project_director,Project_extent_preserved,Project_date_range,Project_URL,hours_preserved
6,2016,"$25,000.00",Radio Preservation Task Force (Library of Cong...,Preservation of the history of public broadcas...,990_2016,990 says NRPF gave $75K this year,NaN,Washington,DC,grant,networking and coordination,conference,Expanding Awareness of the History of U.S. Pub...,"Funding for the Radio Preservation Task Force,...",Josh Shepperd,NaN,NaN,<NA>,NaN
7,2016,"$25,000.00",University of North Carolina,Preservation of the recordings of Mike Seeger,990_2016,NaN,NaN,Chapel Hill,NC,grant,preserve - digitize,digitized audio,Preserving the Field Recordings of Mike Seeger,Funding to digitize approximately 622 hours of...,NaN,NaN,NaN,<NA>,622.0
8,2016,"$25,000.00",Indiana University,"Preservation of ""Orson Welles on the Air""",990_2016,NaN,NaN,Bloomington,IN,grant,preserve - digitize,digitized audio,Preserving <em>Orson Welles on the Air</em>,"Funding to support the digitization of master,...",NaN,NaN,NaN,<NA>,NaN
10,2018,"$15,000.00",WYSO Radio,Document status of recording archives held by ...,990_2018,This was a research report - could have more t...,NaN,Yellow Springs,OH,grant,report,report,Surveying Radio Recording Archives at Historic...,“Nearly a third of Historically Black Colleges...,Jocelyn Robinson,NaN,NaN,<NA>,NaN
11,2018,"$15,000.00",Arhoolie Foundation,Digitize and preserve the Harry Oster Collecti...,990_2018,NaN,NaN,El Cerrito,CA,grant,preserve - digitize,NaN,Digitizing the Harry Oster Collection of Field...,“The funds will allow us to digitally preserve...,NaN,NaN,NaN,<NA>,NaN
12,2018,"$10,000.00",New York Public Radio,Digitize and Upload WNYC's On the Media series,990_2018,NaN,NaN,New York,NY,grant,preserve - digitize,NaN,Digitizing WNYC's <em>On the Media</em>,The funds will allow WNYC Archives to continue...,NaN,NaN,NaN,<NA>,NaN
13,2018,"$10,000.00",Other Minds,"Digitize and preserve the ""Composer to Compose...",990_2018,NaN,NaN,San Francisco,CA,grant,preserve - digitize,NaN,Digitizing the <em>Composer to Composer</em> R...,"Other Minds in San Francisco will digitize, ca...",Charles Amirkhanian,"DAT tapes, 120 hours",1988&ndash;1991,<NA>,NaN
14,2019,"$10,000.00",KUT Radio (University of Texas at Austin),NaN,990_2019,NaN,74-6000203,Austin,TX,grant,preserve - digitize,NaN,Digitizing KUT Radio Collections,"Says Laura Willis, Digital Assets and Operatio...",NaN,NaN,NaN,<NA>,NaN
15,2019,"$15,000.00",World Music Productions,Preservation of recordings of Afropop Worldwide,990_2019,"Afropop noted in documents, not sure this is i...",52-1490194,Brooklyn,NY,grant,preserve - digitize,NaN,Digitizing Recordings of <em>Afropop Worldwide...,Peabody Award-winning Afropop Worldwide had be...,Laura Willis,NaN,NaN,<NA>,NaN
16,2019,"$10,000.00",National Public Radio,NaN,990_2019,Julie Rogers quoted in news release,52-0907625,Washington,DC,grant,preserve - digitize,NaN,Digitizing <em>Voices in the Wind</em>,“Voices in the Wind established NPR’s reputati...,Julie Rogers,NaN,NaN,<NA>,NaN


In [39]:
list(grants.columns)

['Award_Year_Reported_on_990',
 'Amount',
 'Recipient',
 'Description_990',
 'Source',
 'Notes',
 'Recipient_EIN',
 'City',
 'State',
 'Type',
 'Activity',
 'Output',
 'Title_web',
 'Description_web',
 'Project_director',
 'Project_extent_preserved',
 'Project_date_range',
 'Project_URL',
 'hours_preserved']

Next few cells worked out some minor issues with string parsing

In [30]:
grant.Recipient = 'Appalshop (UT Austin)'

In [31]:
grant.Recipient.split('(')[0].strip()

'Appalshop'

In [32]:
type(grant.Recipient)

str

In [33]:
grant.Recipient

'Appalshop (UT Austin)'

Next cell writes the grant entries:

In [41]:
# now, write the grant records, creates a long strong called 'md'
for row, grant in grants.iterrows():
    # make a filename
    slugified_name = grant.Recipient.split('(')[0].strip().lower().replace(',','').replace(' ','-')
    year = grant.Award_Year_Reported_on_990.strip()
    md_filename = str(year + '-' + slugified_name + '.md')

    md = "---\ntitle: \"" + str(grant.Title_web) + "\"\n"
    md += "collection: grants" + "\n"
    md += "type: grant" + "\n"

    md += "amount: \"" + grant.Amount + "\"\n"
    md += "year: " + year + "\n"
    md += "recipient: " + grant.Recipient.strip() + "\n"
     
    md += "location: \"" + grant.City + ", " + grant.State + "\"\n"
    md += "---\n"

    # write the grant info
    md += "\n" + str(grant.Description_web)

    # add a blank line, for markdwon conventions
    md += "\n"

    # write the grant record md file
    md_filename = os.path.basename(md_filename)
    with open("../_grants/" + md_filename, 'w') as f: 
        f.write(md)